In [0]:
import os
import sys
import wave
import timeit; program_start_time = timeit.default_timer()
import random; random.seed(int(timeit.default_timer()))
import numpy as np
import scipy.io.wavfile as wav
import python_speech_features as features
import torch
import torch.optim as optim
import torch.nn as nn
class ManytooneLstm(nn.Module): #model Definition
 def __init__(self,hidden_cell_dim,inp_dim,out_dim,c0,h0):
  super(ManytooneLstm,self).__init__()
  self.hidden_cell_dim=hidden_cell_dim
  self.inp_dim=inp_dim
  self.out_dim=out_dim
  self.lstm=nn.LSTM(self.inp_dim,self.hidden_cell_dim,num_layers=2,batch_first=True)
  self.linear=nn.Linear(self.hidden_cell_dim,self.out_dim)
  self.log_softmax=nn.LogSoftmax(dim=1)
  self.c0=c0
  self.h0=h0
 def forward(self,batch_data):
  rnn_out,_=self.lstm(batch_data,(self.h0,self.c0))
  out=rnn_out[:,-1,:].view(batch_data.shape[0],-1)
  scores=self.log_softmax(self.linear(out))
  return scores
def create_mfcc(filename):
	(rate,sample) = wav.read(filename)

	mfcc = features.mfcc(sample, rate, winlen=0.025, winstep=0.01, numcep = 13, nfilt=26,
	preemph=0.97, appendEnergy=True)
	d_mfcc = features.delta(mfcc, 2)
	a_mfcc = features.delta(d_mfcc, 2)

	out = np.concatenate([mfcc, d_mfcc, a_mfcc], axis=1)

	return out, out.shape[0]
def calc_norm_param(X):
 """Assumes X to be a list of arrays (of differing sizes)"""
 total_len = 0
 mean_val = np.zeros(X[0].shape[1])
 std_val = np.zeros(X[0].shape[1])
 for obs in X:
  obs_len = obs.shape[0]
  mean_val += np.mean(obs,axis=0)*obs_len
  std_val += np.std(obs, axis=0)*obs_len
  total_len += obs_len	
 mean_val /= total_len
 std_val /= total_len
 return mean_val, std_val
def normalize(X, mean_val, std_val):
	for i in range(len(X)):
		X[i] = (X[i] - mean_val)/std_val
	return X
#recording_command="arecord -f S16_LE -r 16000 -d 2 "+time_now+".wav"
#os.system(recording_command)
feats,feats_len=create_mfcc("test.wav")  #For the testing
mean,var=calc_norm_param([feats])
feats=normalize([feats],mean,var)
device = torch.device("cuda:0" if True else "cpu")
h0 = torch.randn(2,1,128).cuda(device)
c0= torch.randn(2,1,128).cuda(device)
model =ManytooneLstm(128,39,10,c0,h0).cuda(device)
model.load_state_dict(torch.load("model30.pt"))
data=torch.from_numpy(feats[0]).view(1,-1,39)
print(data.shape)
predictions=model(data.type(torch.float32).cuda(device))
print(torch.argmax(predictions),predictions)